In [1]:
%load_ext autoreload
%autoreload 2

In [13]:
import os
import pickle
import numpy as np
from pathlib import Path

from src.visualization.render_mp4 import data_to_video, pickle_data, create_mp4_jupyter

In [3]:
os.environ["DATA_DIR_RAW"] = "/data/scratch-oc40/pulkitag/ankile/furniture-data/"

In [4]:
teleopdir = Path(os.environ["DATA_DIR_RAW"]) / "raw" / "sim" / "teleop"

In [28]:
pickles = list(teleopdir.rglob("*success.pkl"))

len(pickles)

61

In [29]:
tmp_rollout_dir = Path(".") / "tmp_rollout_dir"
tmp_rollout_dir.mkdir(exist_ok=True)

In [24]:
total = 0

for i, p in enumerate(pickles):
    data = pickle_data(p)
    if not data["success"]:
        continue
    ims = data_to_video(data)
    video_tag = create_mp4_jupyter(ims, tmp_rollout_dir / f"video{i}.mp4")
    display(video_tag)
    total += 1

print(f"Total successes: {total}")

File saved as tmp_rollout_dir/video0.mp4


File saved as tmp_rollout_dir/video1.mp4


File saved as tmp_rollout_dir/video4.mp4


File saved as tmp_rollout_dir/video5.mp4


File saved as tmp_rollout_dir/video6.mp4


...

File saved as tmp_rollout_dir/video66.mp4


File saved as tmp_rollout_dir/video67.mp4


File saved as tmp_rollout_dir/video68.mp4


File saved as tmp_rollout_dir/video69.mp4


Total successes: 54
